In [2]:
# pip install opencv-python numpy imutils cmake

  Using cached imutils-0.5.4.tar.gz (17 kB)
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25858 sha256=729e12672fdc17dd34e0f88446af49ba245315a8636672e6c746925a62d2fbee
  Stored in directory: c:\users\taeyoonkim\appdata\local\pip\cache\wheels\f5\0c\3a\61b992f7aa85de40f339e6d4970d91dddb103dd0ad6c5d58f2
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [2]:
# !conda install -c conda-forge dlib -y

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(143): Could not remove or rename C:\Users\TaeyoonKim\anaconda3\pkgs\pytorch-1.10.2-py3.8_cuda10.2_cudnn7_0.tar.bz2.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(143): Could not remove or rename C:\Users\TaeyoonKim\anaconda3\pkgs\pytorch-1.10.2-py3.8_cuda10.2_cudnn7_0\Lib\site-packages\torch\lib\kineto.lib.  Please remove this file manually (you may need to reboot to free file handles)



Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\TaeyoonKim\anaconda3\envs\ocr

  added / updated specs:
    - dlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    dlib-19.22.0               |   py36h9f2b841_0         3.5 MB  conda-forge
    intel-openmp-2022.0.0      |    h57928b3_3663         3.6 MB  conda-forge
    jpeg-9e                    |       h8ffe710_0         373 KB  conda-forge
    libblas-3.9.0              |     13_win64_mkl         5.3 MB  conda-forge
    libcblas-3.9.0             |     13_win64_mkl         5.3 MB  conda-forge
    liblapack-3.9.0            |     13_win64_mkl         5.3 MB  conda-forge
    libpng-1.6.37              |       h1d00b33_2         724 KB  conda-forge
    libzlib-1.2.11             |    h8ffe710_1013          63 KB  conda-forge
    mkl-2022.0.0               |     h0e2418a_796       181.9 MB  co

In [2]:
import cv2, dlib, sys
import numpy as np

# resize (이미지 크기를 줄임)
scaler = 0.3

# initialize face detector and shape predictor
detector = dlib.get_frontal_face_detector()  # 얼굴 디텍터 모듈 초기화
# shape_pred.dat은 머신러닝으로 학습된 모델 
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # 얼굴 특징점 모듈 초기화

# load video(동영상 파일 로드)
# VideoCapture(0) - 파일 이름대신 0을 넣으면 웹캠
cap = cv2.VideoCapture('Man3.mp4')
#cap = cv2.VideoCapture(0)


# load overlay image(라이언 이미지)
overlay = cv2.imread('ryan_transparent.png', cv2.IMREAD_UNCHANGED) # 알파채널까지 함께 읽어야 함.(투명)
# print(type(overlay), overlay.shape)    # <class 'numpy.ndarray'> (130, 129, 4) (h, w, c) 알파채널 포함 4
# overlay function
def overlay_transparent(background_img, img_to_overlay_t, x, y, overlay_size=None):
  bg_img = background_img.copy()
  # convert 3 channels to 4 channels
  if bg_img.shape[2] == 3:
    bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGR2BGRA)

  if overlay_size is not None:
    img_to_overlay_t = cv2.resize(img_to_overlay_t.copy(), overlay_size)

  b, g, r, a = cv2.split(img_to_overlay_t)

  mask = cv2.medianBlur(a, 5)

  h, w, _ = img_to_overlay_t.shape
  roi = bg_img[int(y-h/2):int(y+h/2), int(x-w/2):int(x+w/2)]

  img1_bg = cv2.bitwise_and(roi.copy(), roi.copy(), mask=cv2.bitwise_not(mask))
  img2_fg = cv2.bitwise_and(img_to_overlay_t, img_to_overlay_t, mask=mask)

  bg_img[int(y-h/2):int(y+h/2), int(x-w/2):int(x+w/2)] = cv2.add(img1_bg, img2_fg)

  # convert 4 channels to 4 channels
  bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGRA2BGR)

  return bg_img

while True:
    # read frame buffer from video
    ret, img = cap.read()
    if not ret:
        break
    
    # resize frame
    # resize(img, dsize) 크기 조절(꼭 정수형으로 값을 받아야함)
    img = cv2.resize(img, (int(img.shape[1] * scaler), int(img.shape[0] * scaler)))
    ori = img.copy()
    
    # 얼굴 찾기
    faces = detector(img)
    face = faces[0]  # 여러얼굴이 나오니까 첫번째 얼굴만
    
    # 얼굴 특징점(facial landmark) 찾기
    dlib_shape = predictor(img, face)
    shape_2d = np.array([[p.x, p.y] for p in dlib_shape.parts()])  # dlib 객체를 np로 변환
    
    # 얻은 특징점으로 얼굴 size구하기(좌표 구하기)
    # 좌상단 우하단
    top_left = np.min(shape_2d, axis=0)
    bottom_right = np.max(shape_2d, axis=0)
#     print(top_left, bottom_right)      # [246  84] [328 172]
    
    # compute face size
    # 우하단에서 좌상단뺀 x, y길이의 가장 긴 값
    face_size = max(bottom_right - top_left)     # 82 88
#     print(face_size)
    # 라이언 얼굴 사이즈를 키우기 위해 1.8배 
    face_size = int(np.mean(face_size) * 1)
#     print(face_size)
    
    # compute face center
    # 랜드마크로 얼굴의 중심(좌표 구하기)
    # 모든 특징점의 평균을 구하여 얼굴 중심을 구함, 정수형으로 변환필요
    center_x, center_y = np.mean(shape_2d, axis=0).astype(np.int)
    
    # draw overlay on face
    # 라이언이미지를 센터x, y를 중심으로 넣고 overlay사이즈(얼굴의 가로세로중 가장큰 크기)만큼 resize해서 원본이미지에 넣어줌
    result = overlay_transparent(ori, overlay, 
                                 center_x, center_y, 
                                 overlay_size=(face_size, face_size))
    
    
    # draw
    # rectangle(img, 좌상단좌표, 우하단 좌표)
    img = cv2.rectangle(img, 
                        pt1=(face.left(), face.top()),
                        pt2=(face.right(), face.bottom()),
                        color=(255, 255, 255), # 흰색
                        thickness=2,
                        lineType=cv2.LINE_AA
                       )
    # 랜드마크를 원으로 표시
    for s in shape_2d:
        cv2.circle(img, center=tuple(s), radius=1, color=(255, 255, 255), thickness=2, lineType=cv2.LINE_AA)

    # 랜드마크의 좌상단 우하단 크기 그리기
    cv2.circle(img, center=tuple(top_left), radius=1, color=(255, 0, 0), thickness=2, lineType=cv2.LINE_AA)
    cv2.circle(img, center=tuple(bottom_right), radius=1, color=(255, 0, 0), thickness=2, lineType=cv2.LINE_AA)
    cv2.circle(img, center=(center_x, center_y), radius=1, color=(0, 0, 255), thickness=2, lineType=cv2.LINE_AA)


    # visualize
    cv2.imshow('img', img)    # 윈도우 이름과 이미지 파일
    cv2.imshow('result', result)  # 라이언 이미지 오버랩 이미지
    if cv2.waitKey(1) == ord('q'):
        break

IndexError: 

In [3]:
# program Ending.
cap.release()
cv2.destroyAllWindows()